In [4]:
## Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
## Set path
import os
os.chdir('/content/drive/MyDrive/ADM_HW5')

In [6]:
## Import Utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import math
from tqdm import tqdm
from datetime import datetime, date

In [7]:
## Load Data
with open('sx-stackoverflow-a2q.txt') as f:
    a2q = f.readlines()
    f.close()

In [8]:
## Load Data
with open('sx-stackoverflow-c2q.txt') as f:
    c2q = f.readlines()
    f.close()

In [9]:
## Load Data
with open('sx-stackoverflow-c2a.txt') as f:
    c2a = f.readlines()
    f.close()

In [10]:
## Init Graph
G = nx.DiGraph()

In [11]:
## Merge Graph
t_start = datetime(2009, 10, 1)
t_end = datetime(2010, 10, 1)

for line in a2q:
    line = line.replace('\n', '')
    line = line.split(' ') 
    t = datetime.fromtimestamp(int(line[2]))
    t_truncated = date(t.year,t.month, t.day)
    if t_start <= t < t_end :
        if G.has_edge(line[0], line[1]) :
            G[line[0]][line[1]]['weight'] += 0.6
            G[line[0]][line[1]]['timestamp'].append(t_truncated)
        else :
            G.add_edge(line[0], line[1], weight=0.6)
            G[line[0]][line[1]]['timestamp']=[t_truncated]

for line in c2q:
    line = line.replace('\n', '')
    line = line.split(' ') 
    t = datetime.fromtimestamp(int(line[2]))
    t_truncated = date(t.year,t.month, t.day)
    if t_start <= t < t_end :
        if G.has_edge(line[0], line[1]) :
            G[line[0]][line[1]]['weight'] += 0.3
            G[line[0]][line[1]]['timestamp'].append(t_truncated)
        else :
            G.add_edge(line[0], line[1], weight=0.3)
            G[line[0]][line[1]]['timestamp']=[t_truncated]

for line in c2a:
    line = line.replace('\n', '')
    line = line.split(' ') 
    t = datetime.fromtimestamp(int(line[2]))
    t_truncated = date(t.year,t.month, t.day)
    if t_start <= t < t_end :
        if G.has_edge(line[0], line[1]) :
            G[line[0]][line[1]]['weight'] += 0.1
            G[line[0]][line[1]]['timestamp'].append(t_truncated)
        else :
            G.add_edge(line[0], line[1], weight=0.1)
            G[line[0]][line[1]]['timestamp']=[t_truncated]

### **Functionality 2**

In [ ]:
def funct_2(node, time_start, time_end, metric):
    '''
    Input:
        node <- a user/node
        time_start <- start of the interval of time
        time_end <- end of the interval of time
        metric <- one of the following metrics:
                    1. Betweenness
                    2. PageRank
                    3. Closeness Centrality
                    4. Degree Centrality
    '''
    subG = nx.DiGraph()
    edges = list(G.edges)
    for edge in edges:
        if time_start <= G.edges[edge[0], edge[1]]['timestamp'][0] < time_end:
            subG.add_edge(edge[0], edge[1])
            subG[edge[0]][edge[1]]['weight'] = G.edges[edge[0], edge[1]]['weight']
            subG[edge[0]][edge[1]]['timestamp'] = G.edges[edge[0], edge[1]]['timestamp']

    sub_edges = list(subG.edges)

    if metric == 1: # Betweenness centrality
        print('') 
    
    elif metric == 2: # PageRank
        pr = PageRank(subG, 0.25, node)
        print(f'PageRank score of {node}:', pr)

    elif metric == 3: # Closeness centrality
        cc = Closeness(subG, node)
        print(f'Closeneess centrality of {node}', cc) 

    elif metric == 4: # Degree centrality
        count = 0
        for edge in sub_edges:
            if edge[0] == str(node) or edge[1] == str(node):
                count += 1
            if edge[0] == edge[1] == str(node):
                count -= 1

        degree_centrality = count / (len(subG) - 1)
        print(f'Degree centrality of {node}:', degree_centrality)
    
    else:
        print('Insert a valid metric!')

In [17]:
time_start = date(2010, 1, 1)
time_end = date(2010, 1, 3)
subG = nx.DiGraph()
for edge in list(G.edges):
    if time_start <= G.edges[edge[0], edge[1]]['timestamp'][0] < time_end:
        subG.add_edge(edge[0], edge[1])
        subG[edge[0]][edge[1]]['weight'] = G.edges[edge[0], edge[1]]['weight']
        subG[edge[0]][edge[1]]['timestamp'] = G.edges[edge[0], edge[1]]['timestamp']

In [22]:
subG.nodes()

NodeView(('140740', '211662', '226834', '138604', '223541', '22656', '89521', '240698', '241675', '139406', '123349', '191727', '242272', '30132', '22996', '223391', '239806', '8954', '148578', '172211', '23283', '156458', '70345', '30470', '160823', '176873', '156142', '54680', '15168', '75928', '91299', '241563', '25700', '174674', '16292', '203982', '68183', '79909', '46425', '49658', '7412', '83473', '207319', '240931', '58553', '165520', '23354', '37947', '111669', '2766176', '241470', '64904', '12195', '43927', '7714', '56509', '162758', '18309', '81235', '80274', '28896', '56488', '125844', '56495', '19410', '59503', '234371', '48523', '235475', '63756', '189187', '162273', '241763', '172617', '194328', '68920', '12950', '179972', '228936', '239599', '129164', '210754', '31668', '1388192', '82', '221023', '19563', '124797', '56338', '67566', '2731698', '200106', '5056', '8432', '30492', '40347', '214704', '238444', '234018', '25732', '59303', '239168', '24587', '241894', '237438

In [23]:
nx.pagerank(subG, 0.25)['140740']

0.0002393118371791511

In [19]:
def PageRank(G, alpha, node, weight = 'weight'):
    # Create a copy of the graph where the sum of the weights of the edges
    # of each node is equal to one
    W = nx.stochastic_graph(G, weight = weight)
    N = len(G) # Number of nodes

    # Initialize a dictionary of ones / N
    q = dict.fromkeys(W, 1.0/N)

    # Store the starting dictionary
    start_q = q

    # List of nodes with zero edges that point out to them
    nodes = set()
    aa = set()
    for n in W:
        nodes.add(n)
        for _, ign in W.edges(n):
            aa.add(ign)
    ignored_nodes = nodes.difference(aa)

    # Power Iteration: make up to 100 iterations
    for _ in range(100):
        qlast = q # Probability to be at a certain state after t-1 steps
        qt = dict.fromkeys(qlast.keys(), 0) 
        # Score for the ignored nodes
        # Only the damping parameter is considered
        ignored_sum = alpha * sum(qlast[n] for n in ignored_nodes)
        
        for n in qt: # For each node
            # Extract the destination node of the edges and the weight of the edge
            for _, dest_node, wt in W.edges(n, data = weight):
                # Score of the destination node = alpha*q(t-1)
                qt[dest_node] += alpha * qlast[n] * wt 
            # Score of each node = q[0]*alpha + (1 - alpha)*q[0]
            # Total probability to get at the nodes with damping or from another node
            qt[n] += ignored_sum * start_q.get(n, 0) + (1.0 - alpha) * start_q.get(n, 0)
        
        # Check convergence using l1 norm
        err = sum([abs(qt[n] - qlast[n]) for n in qt])
        # Fix tolerance value to check convergence at 0.0001
        if err < N * 0.0001: 
            return qt[node] # Return the PageRank score for the requested node

In [24]:
PageRank(subG, 0.25, '140740')

0.00024123412649959677

In [13]:
def shortest_path(G, source, target):
  shortest_dist  = {} ## records the cost to reach to that node. 
  previous_nodes = {} ## keep track of the path that has led us to this node.
  unvisited_nodes = list(nx.nodes(G)) ## all nodes
  max_dist = math.inf
  
  for node in unvisited_nodes:
    shortest_dist[node] = max_dist ## for all nodes set the distance = + Inf
    previous_nodes[node] = None
  shortest_dist[source] = 0 ## the distance of the source node is 0.

  while len(unvisited_nodes) != 0:
    dist = []
    for node in unvisited_nodes:
      dist.append(shortest_dist[node])
    u = unvisited_nodes[dist.index(np.min(dist))] ## unvisited nodes with dist[u]
    unvisited_nodes.remove(u) ## remove u from unvisited

    ## Consider its neighbors
    for neighbor in G.neighbors(u):
      new_dist = shortest_dist[u] + G[u][neighbor]['weight']
      if shortest_dist[neighbor] > new_dist:
        shortest_dist[neighbor] = new_dist ## update the shortest distance
        previous_nodes[neighbor] = u       ## update the previous node

  ## if the target hasn't a previous node it means that the graph is not connected.
  if previous_nodes[target] == None:
    return ('Not possible')

  path = [target]
  current_node = target
  while current_node != source:
    path.append(previous_nodes[current_node])
    current_node = previous_nodes[current_node]
  path.reverse()

  ## result (Distance and the path)
  return shortest_dist[target], path

In [ ]:
nx.betweenness_centrality(subG)

In [25]:
nx.closeness_centrality(subG, '140740')

0.05490359465936323

In [15]:
def Closeness(G, u):
    '''
    Input:
        - G: a graph
        - u: a node of the graph
    '''
    # Initialize the sum for the shortest path distances
    s = 0

    for node in tqdm(G.nodes):
        # Compute the shortest path for each node
        dist = shortest_path(G, u, node)
        print(dist)
        # If a path exists add its distance to the sum
        if (dist != 'Not possible'):
            s += int(dist[0])

    # Compute the closeness centrality score
    return ((len(G) - 1)/ s)
    

In [26]:
Closeness(subG, '140740')

  0%|          | 1/3451 [00:01<1:06:57,  1.16s/it]

Not possible


  0%|          | 2/3451 [00:02<1:07:19,  1.17s/it]

(0.6, ['140740', '211662'])


  0%|          | 3/3451 [00:03<1:07:25,  1.17s/it]

...

100%|█████████▉| 3450/3451 [1:07:55<00:01,  1.18s/it]

Not possible


100%|██████████| 3451/3451 [1:07:56<00:00,  1.18s/it]

Not possible


0.6134423897581792